**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Istio Canary

## Hello Istio

Very basic traffic management
https://istio.io/blog/2018/incremental-traffic-management/



In [1]:
kubectl label namespace default istio-injection=enabled

namespace/default labeled


## Create a test pod

In [2]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: sleep
  labels:
    app: sleep
spec:
  ports:
  - port: 80
    name: http
  selector:
    app: sleep
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: sleep
spec:
  replicas: 1
  selector:
    matchLabels:
      app: sleep
  template:
    metadata:
      labels:
        app: sleep
    spec:
      containers:
      - name: sleep
        image: pstauffer/curl
        command: ["/bin/sleep", "3650d"]
        imagePullPolicy: IfNotPresent
---
EOF

service/sleep created
deployment.apps/sleep created


In [3]:
export SLEEP_POD=$(kubectl get pod -l app=sleep \
  -o jsonpath={.items..metadata.name})
echo $SLEEP_POD

sleep-56bc8d768-m52vz


In [6]:
kubectl get crd

NAME                                       CREATED AT
authorizationpolicies.security.istio.io    2021-12-01T10:29:47Z
destinationrules.networking.istio.io       2021-12-01T10:29:47Z
envoyfilters.networking.istio.io           2021-12-01T10:29:47Z
gateways.networking.istio.io               2021-12-01T10:29:47Z
istiooperators.install.istio.io            2021-12-01T10:29:47Z
peerauthentications.security.istio.io      2021-12-01T10:29:47Z
requestauthentications.security.istio.io   2021-12-01T10:29:47Z
serviceentries.networking.istio.io         2021-12-01T10:29:47Z
sidecars.networking.istio.io               2021-12-01T10:29:47Z
telemetries.telemetry.istio.io             2021-12-01T10:29:47Z
virtualservices.networking.istio.io        2021-12-01T10:29:47Z
wasmplugins.extensions.istio.io            2021-12-01T10:29:47Z
workloadentries.networking.istio.io        2021-12-01T10:29:47Z
workloadgroups.networking.istio.io         2021-12-01T10:29:47Z


In [4]:
kubectl get destinationrule 

No resources found in default namespace.


In [5]:
kubectl get virtualservice

No resources found in default namespace.


In [7]:
kubectl get deployments

NAME    READY   UP-TO-DATE   AVAILABLE   AGE
sleep   1/1     1            1           57s
web     1/1     1            1           118m
web2    1/1     1            1           113m


In [8]:
kubectl get pods

NAME                    READY   STATUS    RESTARTS   AGE
sleep-56bc8d768-m52vz   2/2     Running   0          63s
web-79d88c97d6-w67s8    1/1     Running   0          118m
web2-5d47994f45-kjqxc   1/1     Running   0          113m


In [9]:
kubectl get pod $SLEEP_POD -o yaml | grep --color=always  -E '$|.*image:.*'

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubectl.kubernetes.io/default-container: sleep
    kubectl.kubernetes.io/default-logs-container: sleep
    prometheus.io/path: /stats/prometheus
    prometheus.io/port: "15020"
    prometheus.io/scrape: "true"
    sidecar.istio.io/status: '{"initContainers":["istio-init"],"containers":["istio-proxy"],"volumes":["istio-envoy","istio-data","istio-podinfo","istio-token","istiod-ca-cert"],"imagePullSecrets":null,"revision":"default"}'
  creationTimestamp: "2021-12-01T10:34:06Z"
  generateName: sleep-56bc8d768-
  labels:
    app: sleep
    pod-template-hash: 56bc8d768
    security.istio.io/tlsMode: istio
    service.istio.io/canonical-name: sleep
    service.istio.io/canonical-revision: latest
  name: sleep-56bc8d768-m52vz
  namespace: default
  ownerReferences:
  - apiVersion: apps/v1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: sleep-56bc8d768
    uid: d007f1fa-c757-4b66-83e7-672494b69e3f
  resourceV

# Admisssion Controlers and Sidecars

- Istio works as an [ Admission Controller](https://kubernetes.io/blog/2019/03/21/a-guide-to-kubernetes-admission-controllers/)
- adding a [Sidecar Pattern](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45406.pdf)
- [Kubernetes Patterns](https://www.redhat.com/cms/managed-files/cm-oreilly-kubernetes-patterns-ebook-f19824-201910-en.pdf)
- [Envoy Proxy](https://www.envoyproxy.io/)
- [Patterns with Envoy Proxy](https://blog.christianposta.com/microservices/00-microservices-patterns-with-envoy-proxy-series/)

In [10]:
kubectl exec $SLEEP_POD -- curl -L heise.de | wc -l

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   229  100   229    0     0    912      0 --:--:-- --:--:-- --:--:--   908
100  706k  100  706k    0     0   254k      0  0:00:02  0:00:02 --:--:--  406k
22149


# Istio Sample Application

In [11]:
cd istio-*

In [12]:
pwd

/minikube-host/notebooks/training-kubernetes-security/istio-1.12.0


In [13]:
kubectl apply -f samples/bookinfo/platform/kube/bookinfo.yaml

service/details created
serviceaccount/bookinfo-details created
deployment.apps/details-v1 created
service/ratings created
serviceaccount/bookinfo-ratings created
deployment.apps/ratings-v1 created
service/reviews created
serviceaccount/bookinfo-reviews created
deployment.apps/reviews-v1 created
deployment.apps/reviews-v2 created
deployment.apps/reviews-v3 created
service/productpage created
serviceaccount/bookinfo-productpage created
deployment.apps/productpage-v1 created


In [29]:
kubectl get pods 

NAME                              READY   STATUS    RESTARTS   AGE
details-v1-79f774bdb9-mxw52       2/2     Running   0          2m34s
productpage-v1-6b746f74dc-w4sxd   2/2     Running   0          2m34s
ratings-v1-b6994bb9-vkwsn         2/2     Running   0          2m34s
reviews-v1-545db77b95-jj7gd       2/2     Running   0          2m34s
reviews-v2-7bf8c9648f-kd5d9       2/2     Running   0          2m34s
reviews-v3-84779c7bbc-b6rgh       2/2     Running   0          2m34s
sleep-56bc8d768-m52vz             2/2     Running   0          18m
web-79d88c97d6-w67s8              1/1     Running   0          136m
web2-5d47994f45-kjqxc             1/1     Running   0          130m


In [18]:
kubectl get services

NAME           TYPE           CLUSTER-IP       EXTERNAL-IP     PORT(S)          AGE
details        ClusterIP      10.109.184.126   <none>          9080/TCP         41s
kubernetes     ClusterIP      10.96.0.1        <none>          443/TCP          25h
nginx-strict   LoadBalancer   10.108.215.42    10.108.215.42   80:31910/TCP     20h
productpage    ClusterIP      10.105.127.36    <none>          9080/TCP         41s
ratings        ClusterIP      10.100.69.38     <none>          9080/TCP         41s
reviews        ClusterIP      10.103.2.253     <none>          9080/TCP         41s
sleep          ClusterIP      10.100.243.9     <none>          80/TCP           16m
web            NodePort       10.109.89.52     <none>          8080:32752/TCP   134m
web2           NodePort       10.102.229.20    <none>          8080:31132/TCP   128m


In [30]:
kubectl exec "$(kubectl get pod -l app=ratings -o jsonpath='{.items[0].metadata.name}')" \
  -c ratings -- curl -s productpage:9080/productpage | grep --color=always -E "$|<title>.*</title>"

<!DOCTYPE html>
<html>
  <head>
    <title>Simple Bookstore App</title>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">

<!-- Latest compiled and minified CSS -->
<link rel="stylesheet" href="static/bootstrap/css/bootstrap.min.css">

<!-- Optional theme -->
<link rel="stylesheet" href="static/bootstrap/css/bootstrap-theme.min.css">

  </head>
  <body>
    
    

<nav class="navbar navbar-inverse navbar-static-top">
  <div class="container">
    <div class="navbar-header">
      <a class="navbar-brand" href="#">BookInfo Sample</a>
    </div>
    
    <button type="button" class="btn btn-default navbar-btn navbar-right" data-toggle="modal" href="#login-modal">Sign
      in</button>
    
  </div>
</nav>

<!---
<div class="navbar navbar-inverse navbar-fixed-top">
  <div class="container">
    <div class="navbar-header pull-left">
      <a class="navbar-brand" href="#">Microservices Fabric Bo

In [31]:
cat samples/bookinfo/networking/bookinfo-gateway.yaml

apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: bookinfo-gateway
spec:
  selector:
    istio: ingressgateway # use istio default controller
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"
---
apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: bookinfo
spec:
  hosts:
  - "*"
  gateways:
  - bookinfo-gateway
  http:
  - match:
    - uri:
        exact: /productpage
    - uri:
        prefix: /static
    - uri:
        exact: /login
    - uri:
        exact: /logout
    - uri:
        prefix: /api/v1/products
    route:
    - destination:
        host: productpage
        port:
          number: 9080


In [32]:
kubectl apply -f samples/bookinfo/networking/bookinfo-gateway.yaml

gateway.networking.istio.io/bookinfo-gateway created
virtualservice.networking.istio.io/bookinfo created


In [33]:
export PATH=$PATH:$(pwd)/bin

In [34]:
istioctl analyze

Warning [IST0103] (Pod default/web-79d88c97d6-w67s8) The pod is missing the Istio proxy. This can often be resolved by restarting or redeploying the workload.
Warning [IST0103] (Pod default/web2-5d47994f45-kjqxc) The pod is missing the Istio proxy. This can often be resolved by restarting or redeploying the workload.
Info [IST0118] (Service default/nginx-strict) Port name  (port: 80, targetPort: 8080) doesn't follow the naming convention of Istio port.
Info [IST0118] (Service default/web) Port name  (port: 8080, targetPort: 8080) doesn't follow the naming convention of Istio port.
Info [IST0118] (Service default/web2) Port name  (port: 8080, targetPort: 8080) doesn't follow the naming convention of Istio port.


In [35]:
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
export SECURE_INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="https")].nodePort}')
export INGRESS_HOST=$(hostname -i)
echo INGRESS_PORT=$INGRESS_PORT
echo SECURE_INGRESS_PORT=$SECURE_INGRESS_PORT
echo INGRESS_HOST=$INGRESS_HOST

INGRESS_PORT=30784
SECURE_INGRESS_PORT=32098
INGRESS_HOST=192.168.49.2


In [36]:
export GATEWAY_URL=$INGRESS_HOST:$INGRESS_PORT
echo "http://$GATEWAY_URL/productpage"

http://192.168.49.2:30784/productpage


In [38]:
kubectl apply -f samples/addons

serviceaccount/grafana created
configmap/grafana created
service/grafana created
deployment.apps/grafana created
configmap/istio-grafana-dashboards created
configmap/istio-services-grafana-dashboards created
deployment.apps/jaeger created
service/tracing created
service/zipkin created
service/jaeger-collector created
serviceaccount/kiali created
configmap/kiali created
clusterrole.rbac.authorization.k8s.io/kiali-viewer created
clusterrole.rbac.authorization.k8s.io/kiali created
clusterrolebinding.rbac.authorization.k8s.io/kiali created
role.rbac.authorization.k8s.io/kiali-controlplane created
rolebinding.rbac.authorization.k8s.io/kiali-controlplane created
service/kiali created
deployment.apps/kiali created
serviceaccount/prometheus created
configmap/prometheus created
clusterrole.rbac.authorization.k8s.io/prometheus created
clusterrolebinding.rbac.authorization.k8s.io/prometheus created
service/prometheus created
deployment.apps/prometheus created


In [42]:
kubectl get pods -n istio-system

NAME                                   READY   STATUS    RESTARTS   AGE
grafana-6ccd56f4b6-dsrmh               1/1     Running   0          96s
istio-egressgateway-7f4864f59c-2gknh   1/1     Running   0          33m
istio-ingressgateway-55d9fb9f-r9zds    1/1     Running   0          33m
istiod-555d47cb65-g8g6c                1/1     Running   0          34m
jaeger-5d44bc5c5d-jdw5n                1/1     Running   0          96s
kiali-79b86ff5bc-xchnp                 1/1     Running   0          95s
prometheus-64fd8ccd65-fp7gq            2/2     Running   0          95s


In [43]:
istioctl dashboard kiali --address $(hostname -i) | sed s/localhost/$(hostname -i)/

http://192.168.49.2:20001/kiali
Failed to open browser; open http://192.168.49.2:20001/kiali in your browser.



## Cleanup

In [44]:
kubectl delete -f samples/addons -f samples/bookinfo/platform/kube/bookinfo.yaml

serviceaccount "grafana" deleted
configmap "grafana" deleted
service "grafana" deleted
deployment.apps "grafana" deleted
configmap "istio-grafana-dashboards" deleted
configmap "istio-services-grafana-dashboards" deleted
deployment.apps "jaeger" deleted
service "tracing" deleted
service "zipkin" deleted
service "jaeger-collector" deleted
serviceaccount "kiali" deleted
configmap "kiali" deleted
clusterrole.rbac.authorization.k8s.io "kiali-viewer" deleted
clusterrole.rbac.authorization.k8s.io "kiali" deleted
clusterrolebinding.rbac.authorization.k8s.io "kiali" deleted
role.rbac.authorization.k8s.io "kiali-controlplane" deleted
rolebinding.rbac.authorization.k8s.io "kiali-controlplane" deleted
service "kiali" deleted
deployment.apps "kiali" deleted
serviceaccount "prometheus" deleted
configmap "prometheus" deleted
clusterrole.rbac.authorization.k8s.io "prometheus" deleted
clusterrolebinding.rbac.authorization.k8s.io "prometheus" deleted
service "prometheus" deleted
deployment.apps "prometh